In [ ]:
!pip install git+https://github.com/afnan47/cuda.git
%load_ext nvcc_plugin

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-9_h0mraw
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-9_h0mraw
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=9fa6267da8cc6d6bb942a256a0c5f8707fd3b0c33abb3cbd72980c429d2683d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-q4si76cz/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%writefile hpc41.cu

#include<iostream>
#include<cuda_runtime.h>

__global__ void addvect(int *A,int *B,int *C,int n)
{
    int i= blockIdx.x *  blockDim.x + threadIdx.x;

    if(i<n)
    {
        C[i]=A[i]+B[i];
    }
}

int main()
{
    int n=100;
    int *A,*B,*C;
    int size=n*sizeof(int);

    cudaMallocHost(&A,size);
    cudaMallocHost(&B,size);
    cudaMallocHost(&C,size);

    for(int i=0;i<n;i++)
    {
        A[i]=i;
        B[i]=i*2;
    }

    int *da,*db,*dc;

    cudaMalloc(&da,size);
    cudaMalloc(&db,size);
    cudaMalloc(&dc,size);

    cudaMemcpy(da,A,size,cudaMemcpyHostToDevice);
    cudaMemcpy(db,B,size,cudaMemcpyHostToDevice);

    int blocksize=256;
    int numblock=(n+blocksize-1)/blocksize;

    addvect<<<numblock,blocksize>>>(da,db,dc,n);

    cudaMemcpy(C,dc,size,cudaMemcpyDeviceToHost);

    for(int i=0;i<10;i++)
    {
        printf("%d + %d = %d \n",A[i],B[i],C[i]);
    }

    cudaFree(da);
    cudaFree(db);
    cudaFree(dc);

    cudaFreeHost(A);
    cudaFreeHost(B);
    cudaFreeHost(C);

    return 0;
}



Writing hpc41.cu


In [ ]:
!nvcc -o hpc41 hpc41.cu -Xcompiler -fopenmp
!./hpc41

0 + 0 = 0 
1 + 2 = 3 
2 + 4 = 6 
3 + 6 = 9 
4 + 8 = 12 
5 + 10 = 15 
6 + 12 = 18 
7 + 14 = 21 
8 + 16 = 24 
9 + 18 = 27 


In [ ]:
#include<stdio.h>
#include<cuda.h>
#define row1 2 /* Number of rows of first matrix */
#define col1 3 /* Number of columns of first matrix */
#define row2 3 /* Number of rows of second matrix */
#define col2 2 /* Number of columns of second matrix */

_global_ void matproduct(int *l,int *m, int *n)
{
    int x=blockIdx.x;
    int y=blockIdx.y;
    int k;
  
n[col2*y+x]=0;
for(k=0;k<col1;k++)
   {
    n[col2*y+x]=n[col2*y+x]+l[col1*y+k]*m[col2*k+x];
   }
}

int main()
{
    int a[row1][col1];
    int b[row2][col2];
    int c[row1][col2];
    int *d,*e,*f;
    int i,j;

    printf("\n Enter elements of first matrix of size 2*3\n");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col1;j++)
            {
                scanf("%d",&a[i][j]);
            }
    }
    printf("\n Enter elements of second matrix of size 3*2\n");
        for(i=0;i<row2;i++)
        {
            for(j=0;j<col2;j++)
                {
                    scanf("%d",&b[i][j]);
                }
        }

    cudaMalloc((void **)&d,row1*col1*sizeof(int));
    cudaMalloc((void **)&e,row2*col2*sizeof(int));
    cudaMalloc((void **)&f,row1*col2*sizeof(int));

 cudaMemcpy(d,a,row1*col1*sizeof(int),cudaMemcpyHostToDevice);
 cudaMemcpy(e,b,row2*col2*sizeof(int),cudaMemcpyHostToDevice);

dim3 grid(col2,row1);
/* Here we are defining two dimensional Grid(collection of blocks) structure. Syntax is dim3 grid(no. of columns,no. of rows) */

    matproduct<<<grid,1>>>(d,e,f);

 cudaMemcpy(c,f,row1*col2*sizeof(int),cudaMemcpyDeviceToHost);
    printf("\nProduct of two matrices:\n ");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col2;j++)
        {
              printf("%d\t",c[i][j]);
        }
        printf("\n");
    }

    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}

Overwriting hpc42.cu


In [ ]:
!nvcc -o hpc42 hpc42.cu -Xcompiler -fopenmp
!./hpc42

Matrix multiplication result:
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 
